# Финальное задание курса "Введение в машинное обучение"

In [1]:
# Загрузим все необходимые модули
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

In [36]:
# Загрузим обучающую выборку в датафрейм пандасов
data = pd.read_csv("data/features.csv", index_col="match_id")
data_size = data.shape[0]

In [5]:
# Удалим признаки, определенные для конца матча
future_features = [
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire',
    'duration'
]
data.drop(future_features, axis=1, inplace=True)

In [38]:
# Посмотрим где есть пропуски
nan_columns = [c for c in data.columns if data[c].count() < data_size]
for c in nan_columns:
    print "{c}: {n} NaNs".format(c=c, n=(data_size - data[c].count()))

first_blood_time: 19553 NaNs
first_blood_team: 19553 NaNs
first_blood_player1: 19553 NaNs
first_blood_player2: 43987 NaNs
radiant_bottle_time: 15691 NaNs
radiant_courier_time: 692 NaNs
radiant_flying_courier_time: 27479 NaNs
radiant_first_ward_time: 1836 NaNs
dire_bottle_time: 16143 NaNs
dire_courier_time: 676 NaNs
dire_flying_courier_time: 26098 NaNs
dire_first_ward_time: 1826 NaNs


Как и было сказано в документации, все пропуски имеют значения - не поставили варды или не купили бутылку, не убили героя до 5 минут матча. Не хотелось бы так "в лоб" заполнять все нулями (ноль имеет значение - это начало матча). Посмотрим какие диапазоны принимают данные фичи:

In [39]:
for c in nan_columns:
    print "{c}: min: {min}, max: {max}".format(
        c=c,
        min=data[c].min(),
        max=data[c].max()
    )

first_blood_time: min: -78.0, max: 300.0
first_blood_team: min: 0.0, max: 1.0
first_blood_player1: min: 0.0, max: 9.0
first_blood_player2: min: 0.0, max: 9.0
radiant_bottle_time: min: -37.0, max: 300.0
radiant_courier_time: min: -90.0, max: 300.0
radiant_flying_courier_time: min: 180.0, max: 300.0
radiant_first_ward_time: min: -236.0, max: 300.0
dire_bottle_time: min: -45.0, max: 300.0
dire_courier_time: min: -90.0, max: 296.0
dire_flying_courier_time: min: 180.0, max: 300.0
dire_first_ward_time: min: -84.0, max: 300.0


С номером команды и героями, участвующими в событии 'first_blood' все ясно. Остальные фичи (кроме 'radiant_first_ward_time' - у него как видим есть выброс в минимуме) принимают значение от -90 (время стартует за полторы минуты до начала матча) до 300 - 5 минут в секундах. Заполним пропущенные значения на малое значение, а также пофиксим выброс (тоже малым значением).

In [40]:
data.fillna(value=-10000.0, inplace=True, axis=1)
data.loc[data['radiant_first_ward_time'] < -90.0, 'radiant_first_ward_time'] = -10000.0

In [41]:
# Посмотрим, остались ли пропуски и какие диапазоны у нас теперь
for c in nan_columns:
    print "{c}: {n} NaNs, min: {min}, max: {max}".format(
        c=c,
        n=(data_size - data[c].count()),
        min=data[c].min(),
        max=data[c].max()
    )

first_blood_time: 0 NaNs, min: -10000.0, max: 300.0
first_blood_team: 0 NaNs, min: -10000.0, max: 1.0
first_blood_player1: 0 NaNs, min: -10000.0, max: 9.0
first_blood_player2: 0 NaNs, min: -10000.0, max: 9.0
radiant_bottle_time: 0 NaNs, min: -10000.0, max: 300.0
radiant_courier_time: 0 NaNs, min: -10000.0, max: 300.0
radiant_flying_courier_time: 0 NaNs, min: -10000.0, max: 300.0
radiant_first_ward_time: 0 NaNs, min: -10000.0, max: 300.0
dire_bottle_time: 0 NaNs, min: -10000.0, max: 300.0
dire_courier_time: 0 NaNs, min: -10000.0, max: 296.0
dire_flying_courier_time: 0 NaNs, min: -10000.0, max: 300.0
dire_first_ward_time: 0 NaNs, min: -10000.0, max: 300.0


In [ ]:
# Выделим целевую переменную и удалим ее из датасета
y = data['radiant_win']
data.drop(['radiant_win'], inplace=True, axis=1)

## Градиентный бустинг в лоб
Отлично, данные подготовлены, можно запустить бустинг, проверить качество по метрике ROC AUC и попытаться найти оптимум по параметру n_estimators.

In [14]:
# Создадим разбиение для кросс-валидации
n_folds = 5
cv = KFold(n=data_size, n_folds=n_folds, shuffle=True)

In [56]:
for n_estim in [10, 20, 30, 100, 200]:
    auc_mean = 0.0
    times = 0.0
    # Будем оценивать кросс-валидацию напрямую, безо всяких GridSearchCV, чтобы показать как это считается
    print "Fitting and calculating quality metric for n={n}".format(n=n_estim)
    for train_idx, test_idx in cv:
        # Создадим классификатор
        clf = GradientBoostingClassifier(n_estimators=n_estim)
        cur_time = time.time()
        # Обучим на 4/5 от трейн-сета, выбранных кросс-валидацией
        clf.fit(data.iloc[train_idx], y.iloc[train_idx])
        times += (time.time() - cur_time)
        # Получим предсказания для 1/5 трейн-сета
        y_pred = clf.predict_proba(data.iloc[test_idx])[:, 1]
        # Посчитаем метрику
        score = roc_auc_score(y.iloc[test_idx], y_pred)
        auc_mean += score
    # Посчитаем среднее для метрики по 5 фолдам, а также среднее затраченного времени    
    auc_mean = auc_mean / float(n_folds)
    times = times / float(n_folds)
    print "Final ROC AUC for {n} = {r}, avg. time: {sec} second(s)".format(n=n_estim, r=auc_mean, sec=times)

Fitting and calculating quality metric for n=10
Final ROC AUC for 10=0.664363197162, took: 10.4882885933 second(s)
Fitting and calculating quality metric for n=20
Final ROC AUC for 20=0.681674512259, took: 19.5691465855 second(s)
Fitting and calculating quality metric for n=30
Final ROC AUC for 30=0.689084320614, took: 29.317814064 second(s)
Fitting and calculating quality metric for n=100
Final ROC AUC for 100=0.70597260216, took: 103.34880228 second(s)
Fitting and calculating quality metric for n=200
Final ROC AUC for 200=0.713664080503, took: 204.339993763 second(s)


### Выводы
Все считалось в 1 поток на ноутбучном i5. Видно, что для n = 10 было затрачено около 10 сек, для n = 20 около 20 сек, для 30 - 30 сек, для 100 - 100 и т.д, то есть время работы алгоритма линейно от носительно параметра n_estimators. Так же видно, что оптимум не достигнут не то, что на n = 30, но даже и на n = 100 (он точно продолжит расти и дальше - я пробовал, хотя очень нелинейно относительно n_estimators).

Значение метрики качества на бустинге для 30 деревьев: **0.68908**.

Как сделать быстрее:
    - Поиграть параметрами модели, использовать меньше деревьев, уменьшить глубину деревьев или learning_rate - это может ухудшить качество.
    
    - Обучаться на меньшем кол-ве данных или признаков.
    
    - Распараллелиться на всех ядрах машины. Тут поможет GridSearchCV (или другие методы или библиотеки поиска гиперпараметров) или библиотека joblib (gridsearch использует ее внутри). Таким образом, каждый прогон каждого разбиения кросс-валидации будет считаться параллельно, если хватит ядер. GridSearchCV довольно глючный, иногда отказывается работать по странным причинам (вроде комментариев на русском языке) и по стек-трейсу ошибки на пяток экранов не всегда можно понять в чем дело. Можно воспользоваться библиотеками или алгоритмами, которые умеют параллелиться автоматически: xgboost библиотека умеет параллелиться сама, RandomForestClassifier из sklearn параллелится с помощью параметра n_jobs.
    
    - Распараллелиться на множестве машин. Закладочка "Clusters" в IPython ноутбуках, кастомные решения, например с помощью библиотеки Celery. Или для джедаев Spark + MlLib на хадуп-кластерах. Под спарк есть питон-расширение pyspark, а так же с помощью SparkQL можно работать с данными в стиле Pandas. 

## Логистическая регрессия
Для нее мы немного по-другому приготовим данные. Они будут готовиться почти так же, как и для бустинга, но вместо малых значений для пропусков я буду использовать средние значения по признаку.

In [9]:
# Заново прочтем данные
data = pd.read_csv("data/features.csv", index_col="match_id")
data_size = data.shape[0]
data.drop(future_features, axis=1, inplace=True)

# Усредним пропуски с учетом колонки с выбросом
data.loc[data['radiant_first_ward_time'] < -90.0, 'radiant_first_ward_time'] = \
    data[data['radiant_first_ward_time'] >= -90.0]['radiant_first_ward_time'].mean()
data.fillna(data.mean(), inplace=True)

# Выделим целевую переменную и удалим ее из датасета
y = data['radiant_win']
data.drop(['radiant_win'], inplace=True, axis=1)

In [109]:
# Обучим сначала как есть (подберем оптимальное значение параметра L2-регуляризации)
for C in np.logspace(-6, 1, 8):
    auc_mean = 0.0
    times = 0.0
    print "Fitting and calculating quality metric for C={c}".format(c=C)
    for train_idx, test_idx in cv:
        clf = LogisticRegression(penalty='l2', C=C)
        cur_time = time.time()
        clf.fit(data.iloc[train_idx], y.iloc[train_idx])
        times += (time.time() - cur_time)
        y_pred = clf.predict_proba(data.iloc[test_idx])[:, 1]
        score = roc_auc_score(y.iloc[test_idx], y_pred)
        auc_mean += score
    auc_mean = auc_mean / float(n_folds)
    times = times / float(n_folds)
    print "Final ROC AUC for {c} = {r}, avg. time: {sec} second(s)".format(c=C, r=auc_mean, sec=times)

Fitting and calculating quality metric for C=1e-06
Final ROC AUC for 1e-06 = 0.513446070187, avg. time: 0.40812458992 second(s)
Fitting and calculating quality metric for C=1e-05
Final ROC AUC for 1e-05 = 0.513446070187, avg. time: 0.418340158463 second(s)
Fitting and calculating quality metric for C=0.0001
Final ROC AUC for 0.0001 = 0.513446070187, avg. time: 0.409064531326 second(s)
Fitting and calculating quality metric for C=0.001
Final ROC AUC for 0.001 = 0.513446070187, avg. time: 0.498295354843 second(s)
Fitting and calculating quality metric for C=0.01
Final ROC AUC for 0.01 = 0.513446070187, avg. time: 0.538801956177 second(s)
Fitting and calculating quality metric for C=0.1
Final ROC AUC for 0.1 = 0.513446070187, avg. time: 0.408539295197 second(s)
Fitting and calculating quality metric for C=1.0
Final ROC AUC for 1.0 = 0.513446070187, avg. time: 0.443279266357 second(s)
Fitting and calculating quality metric for C=10.0
Final ROC AUC for 10.0 = 0.513446070187, avg. time: 0.39

Мда, качество получилось как у тривиального классификатора, дающего в кач-ве ответа всегда 0. Дело в том, что выборки линейно не разделимы, категориальные фичи классификатор использует как числовые, а масштабы фич существенно различаются. Попробуем убрать категориальные фичи (да, вообще), а оставшиеся поскейлить. 

In [10]:
# Для начала посчитаем разных героев
heroes_columns = []
for p in xrange(5):
    heroes_columns.append("r%d_hero" % (p + 1))
    heroes_columns.append("d%d_hero" % (p + 1))
    
heroes_unique = np.unique(data[heroes_columns])
N_unique_heroes = len(heroes_unique)
max_hero_id = heroes_unique.max()
N_unique_heroes, max_hero_id

(108, 112)

Видно, что героев всего 112, но в играх присутствуют только 108 из них.

In [11]:
# Сохраним перед удалением bag-of-heroes в отдельной матричке
# Код взят из задания
X_pick = np.zeros((data_size, max_hero_id))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

# Убираем героев
data.drop(heroes_columns, inplace=True, axis=1)

# Остальные фичи скейлим
data = StandardScaler().fit_transform(data)

In [ ]:
# Обучим классификатор и посчитаем качество на новом датасете
for C in np.logspace(-6, 1, 8):
    auc_mean = 0.0
    times = 0.0
    print "Fitting and calculating quality metric for C={c}".format(c=C)
    for train_idx, test_idx in cv:
        clf = LogisticRegression(penalty='l2', C=C)
        cur_time = time.time()
        clf.fit(data[train_idx], y.iloc[train_idx])
        times += (time.time() - cur_time)
        y_pred = clf.predict_proba(data[test_idx])[:, 1]
        score = roc_auc_score(y.iloc[test_idx], y_pred)
        auc_mean += score
    auc_mean = auc_mean / float(n_folds)
    times = times / float(n_folds)
    print "Final ROC AUC for {c} = {r}, avg. time: {sec} second(s)".format(c=C, r=auc_mean, sec=times)

Fitting and calculating quality metric for C=1e-06
Final ROC AUC for 1e-06 = 0.685809552789, avg. time: 0.409827947617 second(s)
Fitting and calculating quality metric for C=1e-05
Final ROC AUC for 1e-05 = 0.694479360768, avg. time: 0.46659784317 second(s)
Fitting and calculating quality metric for C=0.0001
Final ROC AUC for 0.0001 = 0.712017915524, avg. time: 0.743495559692 second(s)
Fitting and calculating quality metric for C=0.001
Final ROC AUC for 0.001 = 0.716732742203, avg. time: 1.36488780975 second(s)
Fitting and calculating quality metric for C=0.01
Final ROC AUC for 0.01 = 0.71688025897, avg. time: 1.92223472595 second(s)
Fitting and calculating quality metric for C=0.1
Final ROC AUC for 0.1 = 0.716854295076, avg. time: 2.08190789223 second(s)
Fitting and calculating quality metric for C=1.0
Final ROC AUC for 1.0 = 0.716850357714, avg. time: 1.9373673439 second(s)

Намного лучше! Во-первых, мы переплюнули бустинг и нашли оптимум для параметра регуляризации. Теперь лучший результат по метрике ROC AUC равен **0.71688**. И во-вторых линейная регрессия намного быстрее бустинга. Что же будет, если мы добавим к нашему датасету bag-of-heroes?

In [12]:
data = np.hstack((data, X_pick))

In [16]:
# Заново обучим классификатор и посчитаем качество с учетом bag-of-heroes
for C in np.logspace(-6, 1, 8):
    auc_mean = 0.0
    times = 0.0
    print "Fitting and calculating quality metric for C={c}".format(c=C)
    for train_idx, test_idx in cv:
        clf = LogisticRegression(penalty='l2', C=C)
        cur_time = time.time()
        clf.fit(data[train_idx], y.iloc[train_idx])
        times += (time.time() - cur_time)
        y_pred = clf.predict_proba(data[test_idx])[:, 1]
        score = roc_auc_score(y.iloc[test_idx], y_pred)
        auc_mean += score
    auc_mean = auc_mean / float(n_folds)
    times = times / float(n_folds)
    print "Final ROC AUC for {c} = {r}, avg. time: {sec} second(s)".format(c=C, r=auc_mean, sec=times)

Fitting and calculating quality metric for C=1e-06
Final ROC AUC for 1e-06 = 0.687929876055, avg. time: 0.449196243286 second(s)
Fitting and calculating quality metric for C=1e-05
Final ROC AUC for 1e-05 = 0.69832496083, avg. time: 0.535328006744 second(s)
Fitting and calculating quality metric for C=0.0001
Final ROC AUC for 0.0001 = 0.725469529197, avg. time: 0.888973999023 second(s)
Fitting and calculating quality metric for C=0.001
Final ROC AUC for 0.001 = 0.746206428167, avg. time: 1.85397200584 second(s)
Fitting and calculating quality metric for C=0.01
Final ROC AUC for 0.01 = 0.7515877894, avg. time: 3.01976079941 second(s)
Fitting and calculating quality metric for C=0.1
Final ROC AUC for 0.1 = 0.751806638585, avg. time: 4.05457396507 second(s)
Fitting and calculating quality metric for C=1.0
Final ROC AUC for 1.0 = 0.751787310642, avg. time: 4.20827679634 second(s)
Fitting and calculating quality metric for C=10.0
Final ROC AUC for 10.0 = 0.751783993432, avg. time: 4.59412231

Модель стала еще лучше с добавлением bag-of-heroes! Видимо, добавление этих 112 новых признаков сделал набор данных достаточно линейно разделимым, чтобы получить качество лучше, чем у бустинга. Скорость работы все так же куда больше, чем у бустинга. Итак, новое значение метрики ROC AUC для лучшего алгоритма: **0.75181**. Оптимум по параметру регуляризации так же достижим.

## Выводы
Бустинг хорошо работает для данной задачи без какой-либо преподготовки данных (кроме убирания пропусков), однако если данные подготовить (преобразовать численные-категориальные фичи в на самом деле категориальные), то логистическая регрессия показывает существенно лучший результат чем бустинг, причем делает это намного быстрее. 

## Kaggle
Сделаем предикшен тестового датасета с использованием лучшего классификатора по метрике ROC AUC - логичестической регрессии с регуляризацией L2 и параметром C = 0.1

In [20]:
# Обучим выбранный классификатор на всем трейн-сете
clf = LogisticRegression(penalty='l2', C=0.1)
times = time.time()
clf.fit(data, y)
print "Time taken for fit: %f seconds" % (time.time() - times)

Time taken for fit: 5.034702 seconds


In [38]:
# Прочтем датасет, посмотрим на пропуски и диапазоны
data = pd.read_csv("data/features_test.csv", index_col='match_id')
data_size = data.shape[0]

# Будущих фич тут нет, ничего не удаляем

nan_columns = [c for c in data.columns if data[c].count() < data_size]
for c in nan_columns:
    print "{c}: min: {min}, max: {max}".format(
        c=c,
        min=data[c].min(),
        max=data[c].max()
    )

first_blood_time: min: -46.0, max: 300.0
first_blood_team: min: 0.0, max: 1.0
first_blood_player1: min: 0.0, max: 9.0
first_blood_player2: min: 0.0, max: 9.0
radiant_bottle_time: min: -19.0, max: 300.0
radiant_courier_time: min: -90.0, max: 276.0
radiant_flying_courier_time: min: 180.0, max: 300.0
radiant_first_ward_time: min: -85.0, max: 297.0
dire_bottle_time: min: -22.0, max: 300.0
dire_courier_time: min: -90.0, max: 291.0
dire_flying_courier_time: min: 180.0, max: 300.0
dire_first_ward_time: min: -83.0, max: 300.0


In [39]:
# Выбросов нет, просто усредним
data.fillna(data.mean(), inplace=True)

# Сохраним перед удалением bag-of-heroes в отдельной матричке
# Код взят из задания
X_pick = np.zeros((data_size, max_hero_id))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

# Убираем героев
data.drop(heroes_columns, inplace=True, axis=1)

# Остальные фичи скейлим
data_sc = StandardScaler().fit_transform(data)

# Вернем обратно героев
data_sc = np.hstack((data_sc, X_pick))

In [44]:
# Получим предсказанные вероятности класса
y_pred = pd.DataFrame(data=clf.predict_proba(data_sc)[:, 1], index=data.index, columns=['radiant_win'])

In [46]:
# Запишем в csv
y_pred.to_csv('data/submission.csv', header=True, index=True)

In [47]:
# Посмотрим, что записалось
! head data/submission.csv

match_id,radiant_win
6,0.850643326166
7,0.746478855938
10,0.204861783524
13,0.859376059674
16,0.244659652357
18,0.409745211974
19,0.510930975542
24,0.559346792983
33,0.219988610415


Отлично! Отсылаем задание на Кегл и радуемся заслуженному результату ROC AUC **0.75534** и месту **82** (на 12.03.2016 17:24 MSK).
![caption](data/kaggle.png)